<b> The models generated by this code have been saved in the models directory. There is no requirement for running this notebook again. </b>

In [2]:
# import modules & set up logging
import os
import re
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Generate sentences from files located on disk. This code reads files one by one and generates sentences.
# I also do some of the cleaning of special characters but not all. The reason for this is that some of the scientific
# terms are concatenated by some special characters. They have some meaning which would otherwise be lost.

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                line = re.sub(r' \(.+\)','',line) # removes parenthesis brackets
                line = re.sub(r' {.+}','',line)
                line = re.sub(r' [.+]','',line)
                line = line.replace(',','').replace(':','').replace('.','').replace(';','').replace('\'','').replace('"','')
                #line = re.sub(r'[^A-Za-z0-9]+','',line)
                yield line.split()
                
sentences = MySentences('cs') # a memory-friendly iterator

# The wordVec model used here uses min_count = 5 which is pruning parameter which I think ok for this type of corpus as 
# the words are mainly dealing with "Computer Science" and these are technical extracts.  I will do furthur dictionary 
# reduction down the line.  The window=5 is chosen based on above reason.
model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

#
model.save('model/mymodel2')


2018-04-30 17:00:30,415 : INFO : collecting all words and their counts
2018-04-30 17:00:30,466 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-30 17:00:31,417 : INFO : PROGRESS: at sentence #10000, processed 1134915 words, keeping 40339 word types
2018-04-30 17:00:32,379 : INFO : PROGRESS: at sentence #20000, processed 2297074 words, keeping 62439 word types
2018-04-30 17:00:33,354 : INFO : PROGRESS: at sentence #30000, processed 3465753 words, keeping 81038 word types
2018-04-30 17:00:34,402 : INFO : PROGRESS: at sentence #40000, processed 4636114 words, keeping 98138 word types
2018-04-30 17:00:35,536 : INFO : PROGRESS: at sentence #50000, processed 5832441 words, keeping 115024 word types
2018-04-30 17:00:36,660 : INFO : PROGRESS: at sentence #60000, processed 7038266 words, keeping 130724 word types
2018-04-30 17:00:37,867 : INFO : PROGRESS: at sentence #70000, processed 8245216 words, keeping 145478 word types
2018-04-30 17:00:39,018 : INFO : PRO

In [4]:
# generate dictionary from the sentence corpus where every unique term is assigned an index. 
from gensim import corpora

%time dictionary = corpora.Dictionary(sentences)
print(dictionary)


2018-04-30 17:03:27,474 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-04-30 17:03:29,465 : INFO : adding document #10000 to Dictionary(40339 unique tokens: ['illuminates', 'merriam-webster', '$\\hat{t}^{-}=\\{y\\in', 'sum', 'reconfigurations']...)
2018-04-30 17:03:31,459 : INFO : adding document #20000 to Dictionary(62439 unique tokens: ['vcm', 'lambda-mu-t', 'sum', 'reconfigurations', 'irrevocably']...)
2018-04-30 17:03:33,498 : INFO : adding document #30000 to Dictionary(81038 unique tokens: ['vcm', 'lambda-mu-t', 'm3r', 'sum', 'reconfigurations']...)
2018-04-30 17:03:35,520 : INFO : adding document #40000 to Dictionary(98138 unique tokens: ['cancho', 'lambda-mu-t', 'sum', 'joux', 'irrevocably']...)
2018-04-30 17:03:37,579 : INFO : adding document #50000 to Dictionary(115024 unique tokens: ['verification/validation', 'cancho', 'lambda-mu-t', 'sum', 'joux']...)
2018-04-30 17:03:39,659 : INFO : adding document #60000 to Dictionary(130724 unique tokens: ['verificat

CPU times: user 25.6 s, sys: 1.83 s, total: 27.5 s
Wall time: 27.5 s
Dictionary(334939 unique tokens: ['verification/validation', 'cancho', 'vj', '84870', '24915']...)


In [5]:
# ignore words that appear in less than 20 documents or more than 10% documents
dictionary.filter_extremes(no_below=20, no_above=0.1)
print(dictionary)


2018-04-30 17:04:08,547 : INFO : discarding 319682 tokens: [('gct3', 1), ('has', 26142), ('it', 36589), ('the', 121114), ('this', 87121), ('with', 69384), ('a', 111217), ('by', 59586), ('can', 45656), ('is', 93652)]...
2018-04-30 17:04:08,547 : INFO : keeping 15257 tokens which were in no less than 20 and no more than 22679 (=10.0%) documents
2018-04-30 17:04:08,687 : INFO : resulting dictionary: Dictionary(15257 unique tokens: ['vector-valued', '$d$-regular', 'sum', 'object-based', 'potential']...)


Dictionary(15257 unique tokens: ['vector-valued', '$d$-regular', 'sum', 'object-based', 'potential']...)


In [6]:
# generate document term matrix needed for LDA
doc_term_matrix = [dictionary.doc2bow(sent) for sent in sentences]

In [8]:
# I have tried to run this model on my laptop. Since I have graduated I no longer have access to the AWS servers where
# I had run these models previously.

# I attempted 10 topics and 5 passes and the run stopped on my macbook after 5 hours.
# I tried again with 5 topics and 2 passes and unfortunately it did not run to completion.
# Eventually, I was forced to use a single pass. This is not enough as can be seen with the results.


ldamodel = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=10, id2word = dictionary, passes=1)

print(ldamodel.print_topics(num_topics=10, num_words=10))

ldamodel.save('model/lda.model')



2018-04-30 19:38:25,879 : INFO : using symmetric alpha at 0.1
2018-04-30 19:38:25,880 : INFO : using symmetric eta at 0.1
2018-04-30 19:38:25,883 : INFO : using serial LDA version on this node
2018-04-30 19:38:25,902 : INFO : running online (single-pass) LDA training, 10 topics, 1 passes over the supplied corpus of 226790 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2018-04-30 19:38:25,903 : INFO : PROGRESS: pass 0, at document #2000/226790
2018-04-30 19:38:27,317 : INFO : merging changes from 2000 documents into a model of 226790 documents
2018-04-30 19:38:27,330 : INFO : topic #2 (0.100): 0.008*"one" + 0.007*"number" + 0.007*"complexity" + 0.004*"analysis" + 0.004*"where" + 0.004*"between" + 0.004*"set" + 0.004*"communication" + 0.004*"systems" + 0.004*"some"
2018-04-30 19:38:27,332 : INFO : topic #4 (0.100): 0.006*"channel" + 0.006*"network" + 0.005*"quantum" + 0.004*"all" + 0

[(0, '0.016*"graph" + 0.010*"graphs" + 0.009*"algorithms" + 0.008*"number" + 0.008*"time" + 0.007*"set" + 0.006*"problems" + 0.005*"given" + 0.005*"where" + 0.005*"all"'), (1, '0.010*"codes" + 0.009*"energy" + 0.008*"memory" + 0.008*"performance" + 0.008*"code" + 0.005*"power" + 0.004*"storage" + 0.004*"parallel" + 0.004*"than" + 0.004*"over"'), (2, '0.007*"quantum" + 0.006*"functions" + 0.006*"complexity" + 0.005*"function" + 0.004*"game" + 0.004*"games" + 0.004*"bound" + 0.004*"bounds" + 0.004*"one" + 0.004*"embeddings"'), (3, '0.030*"network" + 0.023*"networks" + 0.009*"nodes" + 0.009*"channel" + 0.006*"communication" + 0.006*"information" + 0.005*"rate" + 0.005*"node" + 0.005*"wireless" + 0.005*"scheme"'), (4, '0.008*"method" + 0.007*"performance" + 0.007*"estimation" + 0.006*"optimization" + 0.006*"signal" + 0.006*"matrix" + 0.005*"noise" + 0.005*"sparse" + 0.004*"sampling" + 0.004*"algorithms"'), (5, '0.007*"method" + 0.005*"distribution" + 0.005*"systems" + 0.004*"models" + 0.00

In [10]:
# In order to overcome this problem I used LDAMulticore.
# As can be seen from the results the topic differentiation is much better.
from gensim import corpora, models

ldamodel = models.LdaMulticore(doc_term_matrix, num_topics=10, id2word = dictionary, passes=5)

print(ldamodel.print_topics(num_topics=10, num_words=10))

ldamodel.save('model/lda.multicore.model')

2018-05-01 10:56:14,026 : INFO : using symmetric alpha at 0.1
2018-05-01 10:56:14,028 : INFO : using symmetric eta at 0.1
2018-05-01 10:56:14,032 : INFO : using serial LDA version on this node
2018-05-01 10:56:14,057 : INFO : running online LDA training, 10 topics, 5 passes over the supplied corpus of 226790 documents, updating every 14000 documents, evaluating every ~140000 documents, iterating 50x with a convergence threshold of 0.001000
2018-05-01 10:56:14,063 : INFO : training LDA model using 7 processes
2018-05-01 10:56:14,122 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/226790, outstanding queue size 1
2018-05-01 10:56:14,203 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/226790, outstanding queue size 2
2018-05-01 10:56:14,211 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/226790, outstanding queue size 3
2018-05-01 10:56:14,269 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/226790,

[(0, '0.007*"theory" + 0.007*"models" + 0.006*"logic" + 0.006*"properties" + 0.006*"networks" + 0.005*"between" + 0.005*"systems" + 0.005*"some" + 0.004*"structure" + 0.004*"new"'), (1, '0.020*"network" + 0.014*"networks" + 0.011*"power" + 0.011*"energy" + 0.010*"performance" + 0.008*"wireless" + 0.008*"nodes" + 0.006*"users" + 0.005*"system" + 0.005*"transmission"'), (2, '0.021*"codes" + 0.017*"channel" + 0.011*"coding" + 0.010*"information" + 0.009*"rate" + 0.008*"error" + 0.008*"quantum" + 0.008*"capacity" + 0.008*"code" + 0.007*"scheme"'), (3, '0.014*"graph" + 0.009*"graphs" + 0.009*"number" + 0.007*"set" + 0.007*"time" + 0.006*"problems" + 0.006*"given" + 0.006*"algorithms" + 0.006*"if" + 0.006*"matrix"'), (4, '0.010*"security" + 0.008*"system" + 0.006*"distributed" + 0.006*"systems" + 0.006*"memory" + 0.006*"performance" + 0.006*"attacks" + 0.005*"code" + 0.005*"implementation" + 0.005*"applications"'), (5, '0.011*"method" + 0.009*"algorithms" + 0.006*"space" + 0.006*"time" + 0.0

<b> This type of model on this kind of corpus can be utilized in creating a context based search application. This would especially useful when search for scientific documents. For example. the term network can mean different things in the context of Computer, Neuroscience or Geology. 

When combined with patent data, this tool can be useful in evaluating new patent applications. 

A hybrid model as described here - https://multithreaded.stitchfix.com/blog/2016/05/27/lda2vec/#topic=38&lambda=1&term=

can be used for more relevant word and topic search in scientific documents.

</b>